<a href="https://colab.research.google.com/github/SergeyShchus/OSM/blob/main/Notebook/OSMX_10_OSMnx_%2B_igraph_for_faster_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install osmnx

In [19]:
%%time 
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [20]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon

%matplotlib inline
ox.__version__

'1.1.2'

In [28]:
import operator

!pip install igraph

import igraph as ig
import networkx as nx
import numpy as np
import osmnx as ox

%matplotlib inline
print(ox.__version__)
print(ig.__version__)

weight = "length"

     |████████████████████████████████| 3.1 MB 7.0 MB/s 
1.1.2
0.9.9


In [29]:
# create networkx graph
G_nx = ox.graph_from_place("Piedmont, CA, USA", network_type="drive")
osmids = list(G_nx.nodes)
G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)

# give each node its original osmid as attribute since we relabeled them
osmid_values = {k: v for k, v in zip(G_nx.nodes, osmids)}
nx.set_node_attributes(G_nx, osmid_values, "osmid")

In [30]:
%%time
# convert networkx graph to igraph
G_ig = ig.Graph(directed=True)
G_ig.add_vertices(G_nx.nodes)
G_ig.add_edges(G_nx.edges())
G_ig.vs["osmid"] = osmids
G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())

CPU times: user 3.93 ms, sys: 968 µs, total: 4.89 ms
Wall time: 5.48 ms


In [31]:
assert len(G_nx.nodes()) == G_ig.vcount()
assert len(G_nx.edges()) == G_ig.ecount()

In [32]:
source = list(G_nx.nodes())[0]
target = list(G_nx.nodes())[-1]

In [33]:
%%time
path1 = G_ig.get_shortest_paths(v=source, to=target, weights=weight)[0]

CPU times: user 398 µs, sys: 0 ns, total: 398 µs
Wall time: 406 µs


In [34]:
%%time
path2 = nx.shortest_path(G_nx, source, target, weight=weight)

CPU times: user 1.5 ms, sys: 0 ns, total: 1.5 ms
Wall time: 1.53 ms


In [35]:
assert path1 == path2

In [36]:
%%time
path_length1 = G_ig.shortest_paths(source=source, target=target, weights=weight)[0][0]

CPU times: user 303 µs, sys: 0 ns, total: 303 µs
Wall time: 312 µs


In [37]:
%%time
path_length2 = nx.shortest_path_length(G_nx, source, target, weight)

CPU times: user 1.34 ms, sys: 0 ns, total: 1.34 ms
Wall time: 1.33 ms


In [38]:
assert path_length1 == path_length2

In [39]:
%%time
closeness1 = G_ig.closeness(vertices=None, mode="ALL", cutoff=None, weights=weight, normalized=True)
max_closeness1 = np.argmax(closeness1)

CPU times: user 22.9 ms, sys: 932 µs, total: 23.8 ms
Wall time: 27.8 ms


In [40]:
%%time
closeness2 = nx.closeness_centrality(G_nx, distance=weight, wf_improved=True)
max_closeness2 = max(closeness2.items(), key=operator.itemgetter(1))[0]

CPU times: user 683 ms, sys: 4.24 ms, total: 687 ms
Wall time: 688 ms


In [41]:
# confirm same node has max closeness in both graphs
assert G_nx.nodes[max_closeness2]["osmid"] == G_ig.vs[max_closeness1]["osmid"]